# Online Convex Optimization

Данные будем брать из этого датасета:
https://archive.ics.uci.edu/ml/datasets/spambase

In [107]:
# подключение необходимых библиотек

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cvx
%matplotlib inline

In [108]:
spam_data = pd.read_csv('spambase.data', header=None)

d = len(spam_data.columns) - 1
spam_data.columns = np.arange(0, 58)
print(d)
spam_data.head()

57


,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


Для дальнейшего применения алгоритмов разобьём данные на значения признаков и столбец целевой переменной.

In [109]:
X = spam_data[np.arange(d)].values
y = spam_data[d].values

### Online gradient descent

Для начала, чтобы убедиться в адекватности построенной модели, применим online градиентный спуск.
Запустим его на разных начальных приближениях.

In [ ]:
R = 1.0
G = 0.00001
D = 2.0 * R

M = R

def proection(x):
    if np.linalg.norm(x) <= R:
        return x
    return x / np.linalg.norm(x) * R

def grad(x, w):
    return (np.dot(x, w) + M) / (4.0*M**2) * w

def calc_opt_value(x_list, y_list):
    w = cvx.Variable(d)
    #print(x_list)
    #print(y_list)
    prob = cvx.Problem(cvx.Minimize(sum(y_list - (x_list*w + M) / (2*M))), 
                       [cvx.sum_squares(w) <= R**2])

    result = prob.solve(solver=cvx.SCS, verbose=False, eps=0.1)
    return result

def calc_regret(cur_sum, x_list, y_list):
    return cur_sum - calc_opt_value(x_list, y_list)

def online_gradient_descent(x0, T, regrets=None):
    x = x0
    t = 1
    cur_sum = 0
    x_list = []

    while t < T:
        alpha = D / (G * float(t)**0.5)
        new_x = proection(x - alpha * grad(x, X[t - 1]))
        
        x = new_x
        x_list.append(x)
        cur_sum += (y[t - 1] - (np.dot(X[t - 1], x) + M) / (2*M))**0.5
        if not (regrets is None) and (t == T - 1):
            regrets.append(calc_regret(cur_sum, np.array(x_list), y[:t]))
        t += 1
    return x
    
def try_online_gd(x0):
    regrets = []
    a = online_gradient_descent(x0, len(spam_data), regrets)
    dots = np.dot(X, a)
    predict = (np.sign(dots) + 1) / 2

    success = len(predict[predict == spam_data[57]])
    print('accuracy: ', float(success) / len(spam_data))
    print('regrets: ', regrets)
    
    
try_online_gd(np.zeros(d))
#try_online_gd(np.ones(d) * R / (float(len(spam_data))**0.5))
#try_online_gd(np.hstack(([R], np.zeros(d - 1))))

/home/mike/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in double_scalars


Построим графики посчитанных значений $regret$ в каждом случае.

**Вывод.**

Как можно увидеть, от начального приближения точность алгоритма не зависит.

0.605955227125
